In [118]:
!pip install -q -U kaggle_environments

In [119]:
import pandas as pd
import random

from kaggle_environments import make, evaluate

Describe agents:

In [120]:
%%writefile all_agents.py
import random
#0 - rock
#1 - paper
#2 - scissors

def rock_agent(observation, configuration):
    return 0

def not_rock_agent(observation, configuration):
    return random.randrange(1, configuration.signs)

def paper_agent(observation, configuration):
    return 1

def not_paper_agent(observation, configuration):
    return random.randrange(0, configuration.signs, 2)

def scissors_agent(observation, configuration):
    return 2

def not_scissors_agent(observation, configuration):
    return random.randrange(0, configuration.signs - 1)

def opponent_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

def random_agent(observation, configuration):
    return random.randrange(random.randrange(0,2), configuration.signs)

def copynext_opponent_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

def copyprevious_opponent_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

last_change_lose_action = None
def change_if_lose_agent(observation, configuration):
    global last_change_lose_action
    if observation.step == 0:
        last_change_lose_action = random.randrange(0, configuration.signs)
    elif (observation.lastOpponentAction - last_change_lose_action) in [-2, 0, 1]:
        last_change_lose_action = (observation.lastOpponentAction + 1) % configuration.signs
    return last_change_lose_action

last_change_win_action = None
def change_if_win_agent(observation, configuration):
    global last_change_win_action
    if observation.step == 0:
        last_change_win_action = random.randrange(0, configuration.signs)
    elif (observation.lastOpponentAction - last_change_win_action) in [-1, 2]:
        last_change_win_action = (observation.lastOpponentAction + 1) % configuration.signs
    return last_change_win_action

counter = 0
def queue_agent (observation, configuration):
    global counter
    counter += 1
    if counter % 3 == 0:
        return 0
    elif counter % 3 == 1:
        return 1
    else:
        return 2

agents = {
    "rock_agent": rock_agent,
    "not_rock_agent": not_rock_agent,
    "paper_agent": paper_agent,
    "not_paper_agent": not_paper_agent,
    "scissors_agent": scissors_agent,
    "not_scissors_agent": not_scissors_agent,
    "opponent_agent": opponent_agent,
    "random_agent": random_agent,
    "copynext_opponent_agent": copynext_opponent_agent,
    "copyprevious_opponent_agent": copyprevious_opponent_agent,
    "change_if_lose_agent": change_if_lose_agent,
    "change_if_win_agent": change_if_win_agent,
    "queue_agent": queue_agent
}

Overwriting all_agents.py


In [121]:
import kaggle_environments
#Игра
from all_agents import agents as all_agents
#Словарь с агентами
agents = {}
for agent in all_agents.items():
    agents[agent] = 0
# Заголовок таблицы результатов
print("-----------Результаты:------------")
# Список игроков
player=list(agents)
rank = {}
# Соревнования - каждый играет друг с другом
for j in enumerate(player):
    #Каждый играет со всеми последующими в списке
    #В Раунд выходит агент-игрок под именем player_1, смена которого произойдет после окончания игры с остальными по списку
    player_1=player[j[0]][1]
    if j[0]+1<len(player):
        #Перебор последующих игроков списка(которые буду играть под именем player_2) c игроком player_1
        for i in player[j[0]+1:]:
            player_2=i[1]
            result = evaluate(
              "rps",
            [player_1, player_2],
            configuration={"episodeSteps": 100})
              #Печать рузьтатов
            print(player[j[0]][0],":",result[0][0], "|", i[0],":",result[0][1])
            rank[player[j[0]][0]] = 0
            rank[i[0]] = 0
for j in enumerate(player):
    #Каждый играет со всеми последующими в списке
    #В Раунд выходит агент-игрок под именем player_1, смена которого произойдет после окончания игры с остальными по списку
    player_1=player[j[0]][1]
    if j[0]+1<len(player):
        #Перебор последующих игроков списка(которые буду играть под именем player_2) c игроком player_1
        for i in player[j[0]+1:]:
            player_2=i[1]
            result = evaluate(
              "rps",
            [player_1, player_2],
            configuration={"episodeSteps": 100})
            if result[0][0] > result[0][1]:
                rank[player[j[0]][0]] += 1
            elif result[0][0] < result[0][1]:
                rank[i[0]] += 1

-----------Результаты:------------
rock_agent : 0 | not_rock_agent : 0
rock_agent : -99.0 | paper_agent : 99.0
rock_agent : 48.0 | not_paper_agent : -48.0
rock_agent : 99.0 | scissors_agent : -99.0
rock_agent : -46.0 | not_scissors_agent : 46.0
rock_agent : 0 | opponent_agent : 0
rock_agent : 0 | random_agent : 0
rock_agent : -98.0 | copynext_opponent_agent : 98.0
rock_agent : 99.0 | copyprevious_opponent_agent : -99.0
rock_agent : -97.0 | change_if_lose_agent : 97.0
rock_agent : 99.0 | change_if_win_agent : -99.0
rock_agent : 0 | queue_agent : 0
not_rock_agent : 60.0 | paper_agent : -60.0
not_rock_agent : -35.0 | not_paper_agent : 35.0
not_rock_agent : -47.0 | scissors_agent : 47.0
not_rock_agent : 37.0 | not_scissors_agent : -37.0
not_rock_agent : 0 | opponent_agent : 0
not_rock_agent : 0 | random_agent : 0
not_rock_agent : -25.0 | copynext_opponent_agent : 25.0
not_rock_agent : 29.0 | copyprevious_opponent_agent : -29.0
not_rock_agent : 0 | change_if_lose_agent : 0
not_rock_agent : 

In [122]:
rank_se = pd.Series(rank, name="Рейтинг агентов, исходя из количества выигранных оппонентов").sort_values(ascending=False).to_frame()
rank_se

,"Рейтинг агентов, исходя из количества выигранных оппонентов"
change_if_lose_agent,9
copynext_opponent_agent,7
rock_agent,4
not_rock_agent,4
scissors_agent,4
not_scissors_agent,4
paper_agent,3
not_paper_agent,3
opponent_agent,2
copyprevious_opponent_agent,2


Вывод:
Наиболее результативными являются агенты, использующие "умные стратегии", наряду с этим стабильные результаты продемонстрировали стратегии "одного выбора".